# Check that all DEM Slope.tif files are downloaded

Before processing the DEM_Slope.tif files and splitting them into 1024x768 tifs and jpgs, this program checks that all of the files are downloaded.
It opens an Excel file listing all PA State game lands and cross references the downloaded files. It also generates a list of statements used to call the program to split the DEM_Slope.tif files into smaller tifs and jpgs.
DEM_Slope.tif files represent a PA State Game Land area and have an area number.
Each tif is either a PAN or PAS depending on the coordinate reference system (CRS) used for it. PAN is CRS 32128. PAS is 32129. Some Game Land areas have both PAN and PAS CRS .tifs, such as area 58. The type of CRS for an area is noted in the data_sheet.csv this program creates.

25 July 2020
Files not downloaded yet:
Area:  319  Row:  291

Areas with both PAN and PAS:  33,58,60,84,95,105,108,120,132,137,151,158,168,193,201,212,217,233,257,258,278,284,317,329

Problem files:
254, 264, 290 and 258 PAS only, don't look right and fail to load.   (They all seem to be 2gb files)

In [ ]:
#%%pycodestyle
import pandas as pd
import os

df = pd.read_excel(r'List of State Game Lands.xlsx')

# add the additional fields
df['File'] = ""
df['PAN'] = ""
df['PAS'] = ""
df['Both'] = ""

print("# ------------------------------------")
print("# Split statements for 0_split_tifs_refactored ")

prevarea = ""
areas_with_both_pan_and_pas = ""
data_sheet = "Area,File,PAN,PAS,Both\n"
for cn in range(1, 5):
    directory = "/storage/images/part_"+str(cn)+"/"
    for filename in sorted(os.listdir(directory)):
        # /storage/images/part_1/41blast2demPAS_Slope_DEM.tif",32129)
        # PAN = 32128 PAS = 32129
        panpas = filename[-17:-14]
        crs = ""
        if(panpas == "PAN"):
            crs = "32128"
            panpas_data = "Y,N,N\n"
            pan_data = "Y"
            pas_data = "N"
        else:
            if(panpas == "PAS"):
                crs = "32129"
                panpas_data = "N,Y,N\n"
                pan_data = "N"
                pas_data = "Y"
            else:
                crs = ""
                panpas_data = "N,N,N\n"
                pan_data = "N"
                pas_data = "N"

        area = filename[:3]
        if(area[-1:] == "b"):
            area = area[:-1]
        area_index = df.index[df['State Game Lands'] == int(area)].tolist()
        df.at[area_index, 'File'] = filename

        if(area == prevarea):
            duplicate = ""
            areas_with_both_pan_and_pas = \
                areas_with_both_pan_and_pas + "," + area
            data_sheet = data_sheet[:-6] + "Y,Y,Y\n"
            df.at[area_index, 'PAN'] = "Y"
            df.at[area_index, 'PAS'] = "Y"
            df.at[area_index, 'Both'] = "Y"

        else:
            duplicate = ""
            data_sheet = data_sheet + area + "," + filename + "," + panpas_data
            df.at[area_index, 'PAN'] = pan_data
            df.at[area_index, 'PAS'] = pas_data
            df.at[area_index, 'Both'] = "N"
        prevarea = area
        print(duplicate +
              'split("' + os.path.join(directory, filename) +
              '",' + crs + ")")

print("------------------------------------")
print("Areas with both PAN and PAS ", areas_with_both_pan_and_pas)
print("")

print("------------------------------------")
print("data_sheet.csv")
print(data_sheet)
data_sheet_file = open("data_sheet.csv", "w")
data_sheet_file.write(data_sheet)
data_sheet_file.close()

print("------------------------------------")
df.to_excel("List of State Game Lands_additional.xlsx")
print(df)

print("------------------------------------")
print("Files not downloaded yet:")
for cn in range(1, len(df)):
    if(df.at[cn, 'File'] == ""):
        print("Area: ", df.at[cn, 'State Game Lands'], " Row: ", cn)

In [5]:
!pip install pycodestyle pycodestyle_magic

     |████████████████████████████████| 41 kB 386 kB/s eta 0:00:011
You should consider upgrading via the '/home/student/anaconda3/envs/notebook/bin/python -m pip install --upgrade pip' command.


In [9]:
!pip install flake8
%load_ext pycodestyle_magic

     |████████████████████████████████| 72 kB 1.1 MB/s eta 0:00:011
     |████████████████████████████████| 66 kB 5.6 MB/s  eta 0:00:01
You should consider upgrading via the '/home/student/anaconda3/envs/notebook/bin/python -m pip install --upgrade pip' command.
